In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_part_number = df['Part_Number'].tolist()

print('总数量：' + str(len(list_part_number)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

from io import BytesIO
import PIL
from openpyxl.drawing import image

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for part_number in list_part_number:
    work.put_nowait(part_number)

def crawler():
    global df
    
    while not work.empty():
        part_number = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get('https://apdi.autocaredata.com/items/' + part_number,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                src = html.xpath('//div[@class="carousel-item active"]/div[@class="image"]/img/@src')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                from io import BytesIO
                import PIL
                from openpyxl.drawing import image
                
                # = = = = = = = = = = = = = = =
                
                pic = PIL.Image.open(BytesIO(requests.get(src,
                                                          headers=UA.get_User_Agent_Requests(),
                                                          proxies=Proxy.get_Proxy_Requests()).content)).convert('RGB')
                pic.save('./pic/' + part_number + '.png', 'png')
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//div[@id="collapseBuyersGuideDetail"]/table/tbody/tr')
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for tr in list_tr:
                    make = tr.xpath('./td[1]/text()')[0].strip()
                    model = tr.xpath('./td[2]/text()')[0].strip()
                    engine = tr.xpath('./td[3]/ul/li[2]/text()')[0].strip()
                    year = tr.xpath('./td[4]/text()')[0].strip()
                    list_vehicle.append(make + ' ' + model + ' ' + engine + ' ' + year)

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//div[@id="collapseInterchanges"]/table/tbody/tr')
                
                # = = = = = = = = = = = = = = =
                
                dict_oe = {}
                for tr in list_tr:
                    name = 'OE_' + tr.xpath('./td/text()')[1].strip()
                    number = tr.xpath('./td/text()')[0].strip()

                    if name not in dict_oe:
                        dict_oe[name] = [number]
                    else:
                        dict_oe[name].append(number)
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//div[@class="item_attributes"]/table/tr/td/table/tr')
                
                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_Number': part_number,
                                         'Vehicle': vehicle,
                                         'Src': src}])
                
                for name in dict_oe:
                    df_temp.loc[0, name] = ';'.join(dict_oe[name])

                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./td/text()')[0].strip()[:-1]] = tr.xpath('./th/text()')[0].strip()

                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Number': part_number}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./info_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：364

3310026  <->  [ok] - 剩余数量：344 - 当前时间：10:11:48
3310025  <->  [ok] - 剩余数量：343 - 当前时间：10:11:49
3310011  <->  [ok] - 剩余数量：342 - 当前时间：10:11:51
3310023  <->  [ok] - 剩余数量：341 - 当前时间：10:11:52
3310012  <->  [ok] - 剩余数量：340 - 当前时间：10:11:52
3310004  <->  [ok] - 剩余数量：339 - 当前时间：10:11:52
3310008  <->  [ok] - 剩余数量：338 - 当前时间：10:11:52
3310015  <->  [ok] - 剩余数量：337 - 当前时间：10:11:53
3310016  <->  [ok] - 剩余数量：336 - 当前时间：10:11:53
3310010  <->  [ok] - 剩余数量：335 - 当前时间：10:11:54
3310022  <->  [ok] - 剩余数量：334 - 当前时间：10:11:54
3310009  <->  [ok] - 剩余数量：333 - 当前时间：10:11:54
3310007  <->  [ok] - 剩余数量：332 - 当前时间：10:11:54
3310006  <->  [ok] - 剩余数量：331 - 当前时间：10:11:55
3310003  <->  [ok] - 剩余数量：330 - 当前时间：10:11:55
3310029  <->  [ok] - 剩余数量：329 - 当前时间：10:11:59
3310037  <->  [ok] - 剩余数量：328 - 当前时间：10:12:00
3310031  <->  [ok] - 剩余数量：327 - 当前时间：10:12:00
3310039  <->  [ok] - 剩余数量：326 - 当前时间：10:12:01
3310030  <->  [ok] - 剩余数量：325 - 当前时间：10:12:02
3310035  <->  [ok] - 剩余数量：324 - 当前时间：10:12:02
3310032  <->  [ok] - 剩余数量

3310263  <->  [ok] - 剩余数量：166 - 当前时间：10:14:07
3310261  <->  [ok] - 剩余数量：165 - 当前时间：10:14:08
3310248  <->  [ok] - 剩余数量：164 - 当前时间：10:14:08
3310264  <->  [ok] - 剩余数量：163 - 当前时间：10:14:09
3310265  <->  [ok] - 剩余数量：162 - 当前时间：10:14:10
3310260  <->  [ok] - 剩余数量：161 - 当前时间：10:14:10
3310267  <->  [ok] - 剩余数量：160 - 当前时间：10:14:12
3310266  <->  [ok] - 剩余数量：159 - 当前时间：10:14:12
3310262  <->  [ok] - 剩余数量：158 - 当前时间：10:14:12
3310268  <->  [ok] - 剩余数量：157 - 当前时间：10:14:13
3310269  <->  [ok] - 剩余数量：156 - 当前时间：10:14:14
3310195  <->  [ok] - 剩余数量：155 - 当前时间：10:14:16
3310271  <->  [ok] - 剩余数量：154 - 当前时间：10:14:17
3310274  <->  [ok] - 剩余数量：153 - 当前时间：10:14:17
3310275  <->  [ok] - 剩余数量：152 - 当前时间：10:14:17
3310042  <->  [ok] - 剩余数量：151 - 当前时间：10:14:18
3310276  <->  [ok] - 剩余数量：150 - 当前时间：10:14:18
3310273  <->  [ok] - 剩余数量：149 - 当前时间：10:14:20
3310284  <->  [ok] - 剩余数量：148 - 当前时间：10:14:21
3310281  <->  [ok] - 剩余数量：147 - 当前时间：10:14:21
3310283  <->  [ok] - 剩余数量：146 - 当前时间：10:14:22
3310279  <->  [ok] - 剩余数量：145 - 当前

3310138  <->  [ok] - 剩余数量：0 - 当前时间：10:18:51
3310586  <->  [ok] - 剩余数量：0 - 当前时间：10:19:12
3310730  <->  [ok] - 剩余数量：0 - 当前时间：10:19:14
3310288  <->  [ok] - 剩余数量：0 - 当前时间：10:19:19

爬虫输出中

爬虫结束
